# Backend Integration Using Flask

In [ ]:
# Import neccessary libraries
from flask import Flask, request, render_template
import pandas as pd
import joblib

app = Flask(__name__)

# Load models and scalers
price_model = joblib.load('final_model_listed.pkl')
rent_model = joblib.load('final_model_rent.pkl')
scaler_listed = joblib.load('scaler_listed.pkl')
scaler_rent = joblib.load('scaler_rent.pkl')

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    try:
        zipcode = request.form.get('zipcode')
        bedrooms = request.form.get('bedrooms')
        bathrooms = request.form.get('bathrooms')
        area = request.form.get('area')
        ppsq = request.form.get('ppsq')

        bedrooms = float(bedrooms) if bedrooms else 3
        bathrooms = float(bathrooms) if bathrooms else 2
        area = float(area) if area else 2128
        ppsq = float(ppsq) if ppsq else 223

        input_data = pd.DataFrame([[zipcode, bedrooms, bathrooms, area, ppsq]], 
                                  columns=['Zipcode', 'Bedroom', 'Bathroom', 'Area', 'PPSq'])

        # Scale input data
        input_scaled_listed = scaler_listed.transform(input_data)
        input_scaled_rent = scaler_rent.transform(input_data)

        # Predictions
        predicted_price = price_model.predict(input_scaled_listed)
        predicted_rent = rent_model.predict(input_scaled_rent)

        predicted_price = round(predicted_price[0])
        predicted_rent = round(predicted_rent[0])

        return render_template('index.html', 
                               price=predicted_price, 
                               rent=predicted_rent, 
                               zipcode=zipcode,
                               bedrooms=bedrooms,
                               bathrooms=bathrooms,
                               area=area,
                               ppsq=ppsq)
    except Exception as e:
        return str(e)

if __name__ == '__main__':
    app.run()